<div class="head0">
    <div class="head0__name">
        Basic examples for the Fokker-Planck solver
    </div>
    <div class="head0__note">
        Solution of the multidimensional Fokker-Planck equation by fast and accurate tensor based methods.
    </div>
</div>

In [1]:
import numpy as np

from fpcross import ij, Grid, Model, Solver, SolversCheck

ij()

Start | 10:48AM MSK on Oct 15, 2019 | python 3.7.1    |
-------------------------------------------------------


<div class="head1">
    <div class="head1__name">
        Solve equation, using explicit functions
    </div>
    <div class="head1__note">
        Consider, for example, a 3-dimensional Focker-Planck equation with the zero drift (diffusion equation) and set the corresponding parameters by hands.
    </div>
</div>

<div class="head2">
    <div class="head2__name">
        Parameters
    </div>
    <div class="head2__note">
Consider
$$
    d x = f(x, t) \, dt + S(x, t) \, d \beta,
    \quad
    d \beta \, d \beta^{\top} = Q(t) dt,
    \quad
    x(0) = x_0 \sim \rho(x, 0) = \rho_0 (x),
$$
$$
    \frac{\partial \rho(x, t)}{\partial t} =
        \sum_{i=1}^d \sum_{j=1}^d
            \frac{\partial^2}{\partial x_i \partial x_j}
            \left[ D_{ij}(x, t) \rho(x, t) \right]
        - \sum_{i=1}^d
            \frac{\partial}{\partial x_i}
            \left[ f_i(x, t) \rho(x, t) \right],
    \quad
     D(x, t) = \frac{1}{2} S(x, t) Q(t) S(x, t)^{\top},
$$
where spatial $d$-dimensional ($d \ge 1$) variable $x \in R^d$ has probability density function (PDF) $\rho(x, t)$, $\beta$ is Brownian motion of dimension $q$ ($q \ge 1$, and we assume below that $q = d$), $f(x, t) \in R^d$ is a vector-function, $S(x, t) \in R^{d \times q}$ and $Q(t) \in R^{q \times q}$ are matrix-functions and $D(x, t) \in R^{d \times d}$ is a diffusion tensor.

Let
$$
    Q(t) \equiv I,
    \,
    S(x, t) \equiv \sqrt{2 D_c} I
    \implies
    D(x, t) \equiv D_c I,
$$
and
$$
    d = 3,
    \quad
    x \in \Omega,
    \quad
    \rho(x, t) |_{\partial \Omega} \approx 0,
    \quad
    f(x, t) \equiv 0,
    \quad
    \rho_0(x) = \frac{1}{(2 \pi s)^{\frac{3}{2}}}\exp{\left[-\frac{|x|^2}{2s}\right]}.
$$

It can be shown that the analytic solution is
$$
    \rho(x, t) =
        (2 \pi s + 4 \pi D t)^{-\frac{3}{2}}
        \exp{ \left[
            - \frac
                {
                    |x|^2
                }
                {
                    2  s + 4 D t
                }
        \right] },
$$
and the stationary solution ($t \rightarrow \infty$) is
$$
    \rho_{stat}(x) = 0.
$$
    </div>
</div>

<div class="note">
    Since interpolation is not required for the case of the zero drift ($f \equiv 0$), but our solver calculates it by design, then it is expected to operate much slower than another simple solvers.
</div>

In [3]:
s = 1.
D = 0.5

class Model_(Model):

    def d(self):
        ''' Spatial dimension. '''
        return 3
    
    def D(self):
        ''' Diffusion coefficient. '''
        return D

    def f0(self, X, t):
        ''' Function f(x, t). '''
        return np.zeros(X.shape)

    def f1(self, X, t):
        ''' Function f(x, t) / d x. '''
        return np.zeros(X.shape)

    def r0(self, X):
        ''' Initial condition. '''
        a = 2. * s
        r = np.exp(-np.sum(X*X, axis=0) / a) / (np.pi * a)**1.5
        return r.reshape(-1)

    def rt(self, X, t):
        ''' Exact analytic solution. '''
        a = 2. * s + 4. * D * t
        r = np.exp(-np.sum(X*X, axis=0) / a) / (np.pi * a)**1.5
        return r.reshape(-1)
    
    def with_rt(self):
        return True

<div class="note">
    We can also set stationary solution $r_s(x)$, but in our case it is equal to zero. Since we do not want compare computation results with the zero, we do not define the corresponding function rs(self, x) in the model.
</div>

<div class="head2">
    <div class="head2__name">
        Solution in the dense (NP) format
    </div>
    <div class="head2__note"></div>
</div>

In [4]:
SL = Solver(
    TG=Grid(d=1, n=10, l=[+0., +1.], kind='u'), # Time grid
    SG=Grid(d=3, n=21, l=[-5., +5.], kind='c'), # Spatial grid
    MD=Model_()                                 # Model for equation
)
SL.init()    # Init solver
SL.prep()    # Prepare special matrices
SL.calc()    # Solve equation
SL.TG.info() # Present results from all submodules
SL.SG.info()
SL.FN.info()
SL.info()

Solve:   0%|                                            | 0/9 [00:00<?, ?step/s]

ValueError: Ivalid limits (min should be less of max).

<div class="head2">
    <div class="head2__name">
        Solution in the sparse (TT) format
    </div>
    <div class="head2__note"></div>
</div>

In [8]:
SL = Solver(
    TG=Grid(d=1, n=10, l=[+0., +1.], kind='u'), # Time grid
    SG=Grid(d=3, n=21, l=[-5., +5.], kind='c'), # Spatial grid
    MD=Model_(),                                # Model for equation
    eps=1.E-2,                                  # TT-approximation accuracy
    with_tt=True
)
SL.prep()
SL.calc()
SL.info()

Solve: 100%|█████| 99/99 [00:53<00:00,  1.22step/s, | At T=1.0e+00 : er=3.8e-03]

----------- Solver
Format    : 3D, TT, eps= 1.00e-02 [order=2]
Grid t    : poi =       100, min =    0.0000, max =    1.0000
Grid x    : poi =        21, min =   -5.0000, max =    5.0000
Time sec  : prep = 2.66e-03, calc = 5.36e+01, spec = 2.06e-01
Err real  : 3.76e-03


<div class="head1">
    <div class="head1__name">
        Solve equation, using the predefined model
    </div>
    <div class="head1__note">
        We can set the same equation, using the predefined corresponding model.
    </div>
</div>

In [4]:
MD = Model.select('fpe_3d_drift_zero')
MD.init(s=1., D=0.5)
MD.info(is_comp=True)

Model : fpe_3d_drift_zero      | d r(x, t) / d t = D \Delta r(x, t)
>>>>>>> Description            : Three-dimensional Focker Planck equation with the zero drift


<div class="head2">
    <div class="head2__name">
        Detailed info about model equation
    </div>
    <div class="head2__note">
        We can present the model description in markdown format.
    </div>
</div>

In [6]:
# MD.info()

<div class="head2">
    <div class="head2__name">
        Solution in the sparse (TT) format, using the model
    </div>
    <div class="head2__note"></div>
</div>

In [11]:
SL = Solver(
    TG=Grid(d=1, n=10, l=[+0., +1.], kind='u'),
    SG=Grid(d=3, n=21, l=[-5., +5.], kind='c'),
    MD=MD, eps=1.E-2, with_tt=True
)
SL.init()
SL.prep()
SL.calc()
SL.info()

Solve: 100%|█████| 99/99 [01:46<00:00,  1.79s/step, | At T=1.0e+00 : er=3.8e-03]

----------- Solver
Format    : 3D, TT, eps= 1.00e-06 [order=2]
Grid t    : poi =       100, min =    0.0000, max =    1.0000
Grid x    : poi =        21, min =   -5.0000, max =    5.0000
Time sec  : prep = 2.72e-03, calc = 1.06e+02, spec = 2.10e-01
Err real  : 3.76e-03


<div class="head1">
    <div class="head1__name">
        Solution visualization
    </div>
    <div class="head1__note"></div>
</div>

In [ ]:
# TODO!

<div class="head1">
    <div class="head1__name">
        Solution of the 3-dimensional Ornstein-Uhlenbeck process
    </div>
    <div class="head1__note">
        Let consider more complex example with known stationary solution.
    </div>
</div>

In [13]:
SL = Solver(
    TG=Grid(d=1, n=100, l=[+0., +1.], kind='u'),
    SG=Grid(d=3, n= 21, l=[-3., +3.], kind='c'),
    MD=Model('fpe_oup', d=3, A=np.eye(3)),
    eps=1.E-6, with_tt=True
)
SL.init()
SL.prep()
SL.calc()
SL.MD.info()
SL.TG.info()
SL.SG.info()
SL.FN.info()
SL.info()

Solve: 100%|█████| 99/99 [01:24<00:00,  1.27step/s, | At T=1.0e+00 : es=1.2e-01]

----------- Solver
Format    : 3D, TT, eps= 1.00e-06 [order=2]
Grid t    : poi =       100, min =    0.0000, max =    1.0000
Grid x    : poi =        21, min =   -3.0000, max =    3.0000
Time sec  : prep = 2.14e-03, calc = 8.42e+01, spec = 2.10e-01
Err stat  : 1.19e-01


<div class="end"></div>